<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/pandas/pandas_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Clean Up Pandas Data

Here we should how to clean up Pandas data, in particular what do with about missing data, as well as what to do with invalid data.

Consider a survey.  If you send people a survey you cannot control in all cases what questions they will answer.  And you cannot anticipate what data they might put that is invalid.  For example, they might leave off some data.  Or they might enter a number in a question that is only supposed to be text.

So here we show you how to :

* generate some random data that is purposefully not clean
* drop duplicate rows
* get rid of rows that have missing values
* convert missing values to something else, like a fixed value of the average of all the other values in the colummn
* delete outliers, which are obvious typos.  For example, here we enter some salaries as $1 million while everyone else is around $100,000.  So that's most likely a mistake (since these are employees and not company owners or the CEO).
* apply a custom function to every row to do whatever special checking you want
* check for missing values
* show different ways to check for numbers or strings and how to convert those when they are of the wrong type

In [55]:
import numpy as np
import pandas as pd

import random

mean=10000
std=25



cols = [("name", str), ("income",np.int8), ("education", str),
     ("age", np.int8), ("city",str), ("id", np.int8), ("email", str), ("salary", np.int8),
        ("citizen", ["Y", "N"])]

words = [np.NaN, "", "abc", "def", "ghi", "jkl", "mno", "pqr"]

records = []


for i in range(20):

  data = {}

  for c in cols:

    if c[1] == np.int8:
      if random.randint(0,5)==5:
          data[c[0]] = np.NaN
      else:
          data[c[0]] = abs(int(random.gauss(mean, std)))

    if c[0] == "citizen":
       if random.randint(0,5)==5:
          data[c[0]] = random.randint(0,10)
       else:
          data[c[0]] =c[1][random.randint(0,1)]

    if c[1] == str and c[0] != "citizen":
       data[c[0]] = words[random.randint(0,len(words)-1)]

    if (c[0] == "income") & (random.randint(0,5)==0):
        data[c[0]] = chr(random.randint(65,70))

    if (c[0] == "income") & (random.randint(0,10)==0):
        data[c[0]] = 1000000

    if (c[0] == "age"):
        data[c[0]] = random.randint(20,65)

  records.append(data)

df=pd.DataFrame(records)


df

,name,income,education,age,city,id,email,salary,citizen
0,NaN,9965,def,49,abc,9971.0,NaN,9949.0,Y
1,def,NaN,mno,37,mno,9972.0,mno,9998.0,Y
2,,10009,def,30,ghi,NaN,pqr,9950.0,N
3,,9999,jkl,38,abc,NaN,,10002.0,6
4,def,NaN,pqr,50,,NaN,pqr,10006.0,N
5,abc,9996,def,63,ghi,10004.0,pqr,9982.0,9
6,abc,1000000,def,45,jkl,10022.0,def,9970.0,Y
7,abc,10020,pqr,61,def,10011.0,ghi,9974.0,0
8,pqr,1000000,def,37,,9983.0,,10015.0,N
9,NaN,NaN,pqr,56,mno,9995.0,jkl,10014.0,10


In [56]:
# check if column in series is empty

df['name'].isnull()


0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14     True
15    False
16    False
17    False
18    False
19    False
Name: name, dtype: bool

In [57]:
# draw from mean()

sMean=df['salary'].mean()

df['salary'] = df['salary'].fillna(sMean)

df

,name,income,education,age,city,id,email,salary,citizen
0,NaN,9965,def,49,abc,9971.0,NaN,9949.000000,Y
1,def,NaN,mno,37,mno,9972.0,mno,9998.000000,Y
2,,10009,def,30,ghi,NaN,pqr,9950.000000,N
3,,9999,jkl,38,abc,NaN,,10002.000000,6
4,def,NaN,pqr,50,,NaN,pqr,10006.000000,N
5,abc,9996,def,63,ghi,10004.0,pqr,9982.000000,9
6,abc,1000000,def,45,jkl,10022.0,def,9970.000000,Y
7,abc,10020,pqr,61,def,10011.0,ghi,9974.000000,0
8,pqr,1000000,def,37,,9983.0,,10015.000000,N
9,NaN,NaN,pqr,56,mno,9995.0,jkl,10014.000000,10


In [58]:
# Yes, No columns cannot contain numbers
# NaN is OK as we can get rid of it in a second step plus certain functions like mean()
# will ignore it.  coerce means convert to NaN on error

df['income'] = df['income'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

df['income']

0        9965.0
1           NaN
2       10009.0
3        9999.0
4           NaN
5        9996.0
6     1000000.0
7       10020.0
8     1000000.0
9           NaN
10       9995.0
11       9975.0
12          NaN
13          NaN
14          NaN
15          NaN
16       9989.0
17       9983.0
18      10030.0
19    1000000.0
Name: income, dtype: float64

In [59]:
# drop outliers, too many standard deviations away


mean = df['income'].mean()
std_dev = df['income'].std()

# Define threshold for outliers (e.g., values more than 2 standard deviations away from the mean)
threshold = 2

df.loc[(df['income'] - mean) > 2 * std_dev]



,name,income,education,age,city,id,email,salary,citizen


In [60]:
df.drop(df.loc[(df['income'] - mean) > 2 * std_dev].index, inplace=True)

df

,name,income,education,age,city,id,email,salary,citizen
0,NaN,9965.0,def,49,abc,9971.0,NaN,9949.000000,Y
1,def,NaN,mno,37,mno,9972.0,mno,9998.000000,Y
2,,10009.0,def,30,ghi,NaN,pqr,9950.000000,N
3,,9999.0,jkl,38,abc,NaN,,10002.000000,6
4,def,NaN,pqr,50,,NaN,pqr,10006.000000,N
5,abc,9996.0,def,63,ghi,10004.0,pqr,9982.000000,9
6,abc,1000000.0,def,45,jkl,10022.0,def,9970.000000,Y
7,abc,10020.0,pqr,61,def,10011.0,ghi,9974.000000,0
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
9,NaN,NaN,pqr,56,mno,9995.0,jkl,10014.000000,10


In [61]:
# drop all numbers in Yes/No answer

df['citizen'] = df['citizen'].apply(lambda x: x if isinstance(x,str) else np.NaN)
df['citizen']


0       Y
1       Y
2       N
3     NaN
4       N
5     NaN
6       Y
7     NaN
8       N
9     NaN
10    NaN
11      Y
12      N
13      N
14      N
15      N
16    NaN
17      Y
18      N
19      N
Name: citizen, dtype: object

In [62]:
df

,name,income,education,age,city,id,email,salary,citizen
0,NaN,9965.0,def,49,abc,9971.0,NaN,9949.000000,Y
1,def,NaN,mno,37,mno,9972.0,mno,9998.000000,Y
2,,10009.0,def,30,ghi,NaN,pqr,9950.000000,N
3,,9999.0,jkl,38,abc,NaN,,10002.000000,NaN
4,def,NaN,pqr,50,,NaN,pqr,10006.000000,N
5,abc,9996.0,def,63,ghi,10004.0,pqr,9982.000000,NaN
6,abc,1000000.0,def,45,jkl,10022.0,def,9970.000000,Y
7,abc,10020.0,pqr,61,def,10011.0,ghi,9974.000000,NaN
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
9,NaN,NaN,pqr,56,mno,9995.0,jkl,10014.000000,NaN


In [63]:
# now we are left with only those rows that have all non-Nan columns.  Note that we still
# have some blanks sells.  You would choose the various procedures we have explained above
# to determine how you want to handle this situation.

df.dropna(how='any', inplace=True)
df

,name,income,education,age,city,id,email,salary,citizen
6,abc,1000000.0,def,45,jkl,10022.0,def,9970.000000,Y
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
11,,9975.0,jkl,50,,9961.0,jkl,9991.529412,Y
17,jkl,9983.0,mno,26,jkl,9995.0,,9991.529412,Y
18,ghi,10030.0,abc,51,pqr,10003.0,mno,10044.000000,N
19,jkl,1000000.0,def,27,abc,10020.0,mno,9973.000000,N


In [64]:
# here we operate on a Pandas series, which is a single column.  But since we are
# sending each row one row at a time we are operating on a single row-column combination
# This s is a single value, a scalar.  So we can work with it as we would with any
# Python primitive (meaning built-in type.  Pandas and Numpy are extension of Python into new types.)


def toUpper(s):
    if isinstance(s,str):
      return s.upper()


df['city']=df['city'].apply(toUpper)
df

,name,income,education,age,city,id,email,salary,citizen
6,abc,1000000.0,def,45,JKL,10022.0,def,9970.000000,Y
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
11,,9975.0,jkl,50,,9961.0,jkl,9991.529412,Y
17,jkl,9983.0,mno,26,JKL,9995.0,,9991.529412,Y
18,ghi,10030.0,abc,51,PQR,10003.0,mno,10044.000000,N
19,jkl,1000000.0,def,27,ABC,10020.0,mno,9973.000000,N


In [65]:
# axis = 1 means row
# axis = 0 means column

# the important point to note there is we send the entire row in as a paramters. thus all the columns
# are available to use.  remember to send back the entire row after you have updated any of the columns.

def wholeRow(row):
  if isinstance(row['city'],str):
     row['city'] = row['city'].lower()
     return row


df=df.apply(wholeRow, axis=1)
df

,name,income,education,age,city,id,email,salary,citizen
6,abc,1000000.0,def,45,jkl,10022.0,def,9970.000000,Y
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
11,,9975.0,jkl,50,,9961.0,jkl,9991.529412,Y
17,jkl,9983.0,mno,26,jkl,9995.0,,9991.529412,Y
18,ghi,10030.0,abc,51,pqr,10003.0,mno,10044.000000,N
19,jkl,1000000.0,def,27,abc,10020.0,mno,9973.000000,N


In [66]:
df

,name,income,education,age,city,id,email,salary,citizen
6,abc,1000000.0,def,45,jkl,10022.0,def,9970.000000,Y
8,pqr,1000000.0,def,37,,9983.0,,10015.000000,N
11,,9975.0,jkl,50,,9961.0,jkl,9991.529412,Y
17,jkl,9983.0,mno,26,jkl,9995.0,,9991.529412,Y
18,ghi,10030.0,abc,51,pqr,10003.0,mno,10044.000000,N
19,jkl,1000000.0,def,27,abc,10020.0,mno,9973.000000,N


# Eliminate Duplicates

eliminate duplicates.  the way to do that is to group all columns and sum.  If the count is bigger then one them delete all but one
